<fig>
<img    src="Edits/logos.png"
        width=450
        height=80>
<figcaption>
</figure>

---
# ***Citometría digital y bioinformática aplicada***
---

# **Sección 1: Origen de los datos** 🌐

<fig>
<img    src="Edits/Data.png"
        width=900
        height=200>
<figcaption>
</figure>

## **1.1 - Introducción**

En un proyecto determinado, la aplicación de estrategias de cómputo depende de la **información disponible del sistema (DATOS)** y de los **objetivos** específicos del estudio.

El laboratorio humedo es el primer acercamiento que tenemos a la generación de datos, mejor conocidos como: **Datos Crudos**. 


---
Datos Crudos: Registros originales tal como se recopilaron en el laboratorio sin ningún procesamiento, filtrado o interpretación.

---

Al trabajar con muestras biológicas diversas, es común analizar:


> *   🩸 Sangre/plasma (biomarcadores)
> *   🔬 Tejidos (biopsias)
> *   🧫 Cultivos celulares (hepatocitos, líneas cancerosas)
> *   🦠 Aislamientos microbianos (resistencia antimicrobiana)



Después de generar los datos, estos pasan a la computadora para proceder a su manejo y adaptarlas acorde a las herramientas de uso.

Las herramientas a utilizar es seleccionada basada a la pregunta del proyecto **¿Que me interesa conocer de mis datos?** 


### Beneficios:


1.   Dejar atrás los análisis unigénicos (qPCR) para adoptar enfoques multiómicos.

2.   Vincular sus muestras a mecanismos moleculares con impacto en terapias y diagnósticos


<fig>
<img    src="Edits/bioinfprocess.png"
        width=600
        height=400>
<figcaption>
</figure>

## 1.2 - Datos transcriptómicos

<fig>
<img    src="Edits/data_trans.jpg"
        width=1000
        height=510>
<figcaption>
</figure>

Nos ayudan a analizar los niveles de expresión génica en diferentes células para inferir **tipos celulares** y **funciones**

| 🔬 **Citometría de Flujo** | 🧪 **Análisis Transcriptómico** |
|-------------------------|-----------------------------|
| Usa anticuerpos y fluorocromos para identificar proteínas de membrana o intracelulares. | Usa secuenciación de ARN para medir la expresión génica. |
| Analiza células individualmente en tiempo real. | Puede analizar muestras bulk (mezcla de células) o datos de single-cell. |
| Limitado por el número de fluorocromos disponibles. | Puede detectar miles de genes simultáneamente. |



**Ejemplo:** En inmunología, podemos usar citometría de flujo para contar y caracterizar células T reguladoras en una muestra, pero con transcriptómica podemos analizar qué genes están expresando, lo que nos permite entender mejor su función y activación.

>	El análisis transcriptómico complementa la citometría de flujo, pues **ayuda a diseñar mejores paneles de anticuerpos y marcadores 🎯.**


## **1.3 - Herramientas Computacionales**

Son programas, plataformas o **algoritmos diseñados para analizar, visualizar o interpretar datos biológicos complejos** (genéticos, químicos, farmacológicos) de manera automática, rápida y precisa. 

**FUNCIÓN:** simplificar tareas que manualmente serían imposibles o muy lentas.

**CONSEJOS:**
> Empieza con herramientas web (CIBERSORTx, STRING) antes de usar software local.

> Usa datos de ejemplo: Muchas herramientas ofrecen datasets de prueba para practicar.

> Busca tutoriales en YouTube: Hay guías visuales para cada herramienta.


**EJEMPLOS:**

> STRING (Web)
Función: Predecir interacciones entre proteínas 

> CIBERSORTx (Web)
Función: Estimar proporciones de tipos celulares en una muestra

> KEGG mapper (Web)
Función: Mapear genes o fármacos en vías bioquímicas conocidas

> hdWGCNA  (Software)
Función:  Encontrar grupos de genes que actúan juntos (módulos) en procesos biológicos.

# **Sección 2: BioComputo** 🧪💻



<fig>
<img    src="Edits/Methodology.png"
        width=400
        height=400>
<figcaption>
</figure>

## **2.1- Google Colaboratory**

<fig>
<img    src="Edits/colab.jpg"
        width=225
        height=125>
<figcaption>
</figure>

#### Iniciación de Google Drive

In [ ]:
from google.colab import drive #Montar Google Drive
drive.mount('/content/drive')

## **2.2 - Preprocesamiento de datos**

## 2.2.1 - scRNA-seq

In [2]:
# Librerias

import pandas as pd
import numpy as np
import umap
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
!pip install scanpy

In [4]:
import scanpy as sc

In [5]:
import anndata

In [ ]:
# Cargar datos
singlecell=pd.read_csv("C:/Users/Rocio/Desktop/singlecell_segmented.csv", index_col=0, header=0) #Single Cell con 3,000 muestras celulares de 15 tipos celulares
#---------- CAMBIAR RUTA ----------

singlecell=singlecell.transpose()
singlecell.head()

In [7]:
# Separar la columna 'cell_types' de la matriz de expresión
cell_types = singlecell['cell_types']  # Guardamos los tipos celulares
singlecell = singlecell.drop(columns=['cell_types'])  # Quitamos la columna de la matriz 

In [8]:
# Separar la fila "feature_name" de la matriz de expresión
feature_name = singlecell.loc['feature_name'].values
singlecell = singlecell.drop(index="feature_name")

In [9]:
# Convertir valores de expresión a float
singlecell = singlecell.astype(float)

## 2.2.2 - Visualización de Datos    

### Uso de Scanpy para el preprocesamiento 

En el análisis de transcriptómica de células individuales (single-cell RNA sequencing, scRNA-seq), trabajamos con grandes volúmenes de datos altamente dimensionales. Para poder extraer información útil, es necesario aplicar técnicas de preprocesamiento, reducción de dimensionalidad, identificación de grupos celulares y visualización.

Scanpy es una de las herramientas más eficientes y ampliamente utilizadas en Python para analizar datos de scRNA-seq. 

Pipeline:

 1️⃣ Leer y organizar los datos de expresión 

 2️⃣ Filtrar células y genes con baja expresión

 3️⃣ Normalizar y aplicar log-transform

 4️⃣ Identificar genes más variables 

 5️⃣ Escalar datos de expresión 

 6️⃣ Construir representaciones de los datos con UMAP para visualizar los grupos celulares.


En caso que posteriormente queramos usar R, Scanpy crea archivo compatibles con Seurat (R), para ellos construye un objeto AnnData que sigue el siguiente formato:

<fig>
<img    src="Edits/anndata_schema.svg"
        width=512
        height=512>
<figcaption>
</figure>

In [ ]:
# Construir el objeto AnnData
adata = anndata.AnnData(X=singlecell.values)

# Asignar nombres correctos a filas y columnas
adata.obs_names = singlecell.index  # Células
adata.var_names = singlecell.columns  # Genes
adata.obs["cell_types"] = cell_types  # Agregar tipos celulares como metadatos
adata.var["feature_name"] = feature_name

# Verificar estructura
print(adata)

In [ ]:
# Filtrar células y genes con baja expresión
sc.pp.filter_cells(adata, min_genes=200)  # Células con al menos 200 genes detectados
sc.pp.filter_genes(adata, min_cells=3)  # Genes expresados en al menos 3 células
adata


In [20]:
# Normalizar la expresión génica (CPM-like normalization)
sc.pp.normalize_total(adata, target_sum=1e4)

# Aplicar log-transformación
sc.pp.log1p(adata)

min_mean=0.0125 → Excluye genes con una expresión promedio muy baja, porque suelen ser ruido.

max_mean=3 → Excluye genes con expresión excesivamente alta, que pueden ser housekeeping genes o artefactos.

min_disp=0.5 → Selecciona genes con alta dispersión (variabilidad relativa), lo que significa que su expresión varía entre células y no es uniforme.

In [ ]:
# Identificar los genes más variables
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var["highly_variable"]]  # Filtrar solo genes variables
adata

In [22]:
sc.pp.scale(adata, max_value=10)

C:\Users\Rocio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scanpy\preprocessing\_scale.py:317: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, metric="euclidean")
sc.tl.umap(adata, min_dist=0.6)
sc.pl.umap(adata, color="cell_types")

In [ ]:
# Extraer la matriz de expresión desde adata.X y convertirla en DataFrame
df_expression = pd.DataFrame(
    adata.X.T,  # Transponemos para que los genes sean filas y las células columnas
    index=adata.var["feature_name"],  # Nombres de los genes como filas
    columns=adata.obs_names  # Nombres de las células como columnas
)
df_expression.head()

In [25]:
df_expression.to_csv("C:/Users/Rocio/Desktop/matriz_expresion_preprocesada.txt", sep="\t", index=True, header=True)

In [26]:
# Guardar como un .h5ad para después convertirlo en .rds en R (opcional)
adata.write("C:/Users/Rocio/Desktop/matriz_expresion_preprocesada.h5ad")

## 2.2.3 - RNA-seq

### Lectura de datos

### Procesamiento de datos RNA-seq

In [ ]:
# Carga de datos
rnaseq=pd.read_csv("/Users/gusano2398gmail.com/Desktop/Prueba/RNA-seq.csv") # Dirección en la que se encuentra el archivo RNA-seq
#---------- CAMBIAR RUTA ----------

print("Base de datos \n")

rnaseq = rnaseq.rename({'gene_symbol': 'Brain Samples', 'b': 'Y'}, axis=1) # Cambiamos de nombre una columna
rnaseq.head() # Mostrar los datos en forma de tabla solo los primeros 5 elementos


### Filtrado de datos

In [ ]:
ages = ["8 pcw", "40 yrs"] #Edades en las que quieres inferir la proporción celular


# Filtara el DF
filtered_df = rnaseq[rnaseq["Age"].isin(ages)]

# Eliminar columna de edad
filtered_df=filtered_df.drop("Age", axis=1)

# Mostrar los resultados
filtered_df=filtered_df.set_index(["Brain Samples"])
print("Base de datos con Indice\n")
print(filtered_df)

filtered_df

# Define the Google Drive path
save_path = "/Users/gusano2398gmail.com/Desktop/Prueba/"

#---------- CAMBIAR RUTA ----------

#Exportar los datos a un archivo tipo csv
filtered_df.to_csv(save_path+"RNA-seq-Filtered.csv")

### Guardar datos en formato txt

In [ ]:
# Carga de datos previamente segmentados
rnaseq=pd.read_csv(save_path+"RNA-seq-Filtered.csv", header=None)
print("Base de datos \n")
rnaseq=rnaseq.transpose()
# Save the file
np.savetxt(save_path+"Bulk.txt", rnaseq, fmt='%s', delimiter='\t')

### Visualización de datos segmentados finales

In [ ]:
rnaseq=pd.read_csv(save_path+"Bulk.txt", header=None, delimiter="\t", index_col=0)
rnaseq

## **2.3 - CIBERSORTx** 📊

<fig>
<img    src="Edits/Methodology.png"
        width=400
        height=400>
<figcaption>
</figure>

### Link a CIBERSORTx: https://cibersortx.stanford.edu/

Pasos a seguir


1.   Crear cuenta
2.   Iniciar sesíon


## 2.3.1 - Firmas genéticas

<fig>
<img    src="Edits/Firmas.png"
        width=800
        height=600>
<figcaption>
</figure>

## 2.3.2 - Inferencia Celular

<fig>
<img    src="Edits/Proporcion.png"
        width=800
        height=600>
<figcaption>
</figure>

<fig>
<img    src="Edits/CIBERSORTx.png"
        width=700
        height=400>
<figcaption>
</figure>

## **2.4 - hdWGCNA** 🕸️

hdWGCNA es una herramienta bioinformática utilizada aplicada en análisis de redes de coexpresión génica en estudios de transcriptómica que nos permite:

- **Identificar grupos de genes que trabajan juntos en diferentes condiciones**, como en enfermedades o en respuesta a fármacos

- **Encontrar biomarcadores y** posibles** dianas terapéuticas** analizando qué genes están más conectados en una red

- Comprender cómo un fármaco afecta la expresión génica y predecir posibles efectos adversos

- Estudiar enfermedades complejas como el cáncer o trastornos neurológicos a nivel molecular


Website: https://smorabit.github.io/hdWGCNA/  

<fig>
<img    src="Edits/Module.png"
        width=400
        height=400>
<figcaption>
</figure>

<fig>
<img    src="Edits/ModuleCoexpression.png"
        width=400
        height=400>
<figcaption>
</figure>

## **2.5 - STRING**

**STRING** (Search Tool for Retrieval of Interacting Genes/Proteins) es una plataforma web gratuita que **permite estudiar interacciones funcionales entre proteínas, genes o compuestos químicos**. Combina datos experimentales, predicciones computacionales y minería de textos para construir redes biológicas.

**Enlace: https://string-db.org**

<fig>
<img    src="Edits/STRING.png"
        width=900
        height=400>
<figcaption>
</figure>

### Análisis avanzado (sin programar)

<fig>
<img    src="Edits/stringview.png"
        width=900
        height=400>
<figcaption>
</figure>

## **2.6- STITCH** 

STITCH es una base de datos de **interacciones proteína-químicos** que integra diversas fuentes de evidencia experimental y que permite seleccionar manualmente con información de minería de texto y predicciones de interacción.


<fig>
<img    src="Edits/STITCH.png"
        width=900
        height=400>
<figcaption>
</figure>

Website: http://stitch.embl.de/ 

# **Sección 3: Ciencia de Datos** 🧬💻

## 5.1- Interpretación de Resultados

<fig>
<img    src="Edits/Visualization.png"
        width=650
        height=400>
<figcaption>
</figure>

.
.
.
.

La interpretación de resultados en biología computacional es el proceso de traducir datos numéricos o estadísticos en conocimiento biológicamente relevante, vinculándolos con mecanismos celulares, enfermedades o aplicaciones farmacéuticas. Se realiza en varias etapas:

**> Validación técnica:**

Confirmar que los resultados son robustos (ej.: repetir análisis con distintos parámetros o subconjuntos de datos).


**> Contextualización biológica:**

Relacionar genes, proteínas o vías identificadas con funciones conocidas usando bases de datos como KEGG, GO o Reactome.

**> Integración con literatura:**

Comparar resultados con estudios previos

**> Herramientas de apoyo:**

**Anotación funcional:** DAVID, Enrichr para identificar vías enriquecidas.

**Redes biológicas:** Cytoscape para visualizar interacciones entre genes/proteínas descubiertos.

**Plataformas farmacológicas:** DrugBank para cruzar dianas identificadas con fármacos existentes.

**> Traducción a aplicaciones prácticas:**

Generar hipótesis experimentales




*¿Qué significa esto para la enfermedad o el fármaco que estudio?*

<fig>
<img    src="Edits/figure.png"
        width=500
        height=700>
<figcaption>
</figure>

<fig>
<img    src="Edits/Trends.png"
        width=700
        height=400>
<figcaption>
</figure>

<fig>
<img    src="Edits/layers_down.png"
        width=700
        height=600>
<figcaption>
</figure>

<fig>
<img    src="Edits/hdWGCNA_string.png"
        width=1020
        height=550>
<figcaption>
</figure>

# **Laboratorio de Innovación Biodigital**

<fig>
<img    src="Edits/lab.jpg"
        width=600
        height=400>
<figcaption>
</figure>

**Algunos de los temas y herramientas que trabajamos en el laboratorio incluyen:**

**Herramientas**
- 🔹 **Programación en R y Python** - Visualización de datos y análisis
- 🔹 **hdWGCNA** – Análisis de redes de coexpresión genética
- 🔹 **Panther y STRING** – Herramientas para el análisis de proteínas y la identificación de interacciones moleculares.
- 🔹 **AlphaFold y RosettaFold** – Modelado de estructuras de proteínas para estudiar sus funciones biológicas.
- 🔹 **IAs: ML y DL** – Predicción de patrones en grandes datasets

---



Si están interesados en **colaborar con nosotros** o en aprender más sobre **análisis de datos biomédicos avanzados**, estamos **ofreciendo cursos especializados** en estos temas. Además, si su laboratorio o equipo tiene proyectos en los que podamos aportar con nuestras herramientas y conocimientos, no duden en ponerse en contacto.


Para más información o consultas, no duden en contactarnos. 